# How to Prove You’re Right: A/B Testing with SciPy
### by Hillary Green-Lerman, Lead Data Scientist, Looker
### Michoel Snow, Senior Data Scientist, Bark
### PyData NYC 2019

## What’s an A/B Test and why do I need it?

How might Amazon use an A/B Test?

How might Netflix use an A/B Test?

How might _you_ use an A/B Test?

## What do I need to run an A/B Test?

Let's start by importing `pandas` (which we'll be using to load our datasets for this tutorial) and `matplotlib` (which we'll be using for graphing).

In [ ]:
import pandas as pd

from matplotlib import pyplot as plt

### Example 1

You've sent out a newsletter to your users with an exciting new offer.  You're not sure what the best subject line is to get recipients to open the email and click through to your website, so you're running an A/B Test.  Your data is in the CSV file `ab_test_dataset_1.csv`.

- What kind of framework might you be using to run this test?

- What are some good metrics to track?

- Is anything missing from this dataset?

In [ ]:
df1 = pd.read_csv('ab_test_dataset_1.csv')

### Example 2
You've just launched a new feature for your website.  You want to know what pop-up message is best to prompt people to use this feature, so you're running an A/B Test.  Your data is in the CSV file `ab_test_dataset_2.csv`.

- What kind of framework might you be using to run this test?

- What are some good metrics to track?

- Is anything missing from this dataset?

In [ ]:
df2 = pd.read_csv('ab_test_dataset_2.csv')

### Example 3
You're considering changing the price of your product.  In order to test how price point affects purchase rates, you randomly send out different discount codes to users for either 5%, 10%, or 20% off of your base price.  Your data is in the CSV file `ab_test_dataset_3.csv`.

- What kind of framework might you be using to run this test?

- What are some good metrics to track?

- Is anything missing from this dataset?

- **Bonus 1:** Why might discount codes not give you a good sense of purchase rates for different price points?

- **Bonus 2:** If the purchase rate at a 5% discount is 0.02 and the purchase rate at a 20% discount is 0.021, should you sent your price point to 20% off of your current base price? *(assume that the difference is significant)*

In [ ]:
df3 = pd.read_csv('ab_test_dataset_3.csv')

## How do I analyze an A/B Test?

### Example 4
Let's prove to ourselves that A/B Test results can be deceiving!

We're going to generate some results drawn from real distributions and see how effect size and sample size affect our ability to get reliable results.

We'll use a seperate `np.random.binomial()` to generate the samples for each variant.  This function takes as arguments:
- The number of samples in each experiment (i.e., how many people would see a given variant of your A/B Test)
- The probability of a success (i.e., the click-through rate for one of your variants)
- The number of experiments to run (we choose a large number so that we can get a reliable estimate)

In [ ]:
import numpy as np

def calc_percent_correct(sample_size, ctr_A, ctr_B, number_experiments):
    '''
    This function performs a set of mock A/B Tests for two click through rates
    and a given sample size and reports the fraction of experiments where the
    directionality of the result is correct.
    
    You can use this function to see how often we would actually
    observe the "correct" results for given conditions.
    '''
    # Create experiments
    experimental_ctr_A = np.random.binomial(sample_size, ctr_A, size=number_experiments) / sample_size
    experimental_ctr_B = np.random.binomial(sample_size, ctr_B, size=number_experiments) / sample_size

    # Calculate the fraction of experiments with "correct" results
    if ctr_A < ctr_B:
        return np.mean(experimental_ctr_A < experimental_ctr_B)
    elif ctr_A > str_b:
        return np.mean(experimental_ctr_A > experimental_ctr_B)
    else:
        return np.mean(experimental_ctr_A == experimental_ctr_B)


Use the function with different values of `ctr_A`, `ctr_B`, and `sample_size`.  How do these affect the fraction of experiments with correct directionality?

In [ ]:
# The parameters for our experiment
sample_size = ###
ctr_A = ###
ctr_B = ###
number_experiments = ###

# How often is the CTR for B greater than for A?
fraction_correct = calc_percent_correct(sample_size, ctr_A, ctr_B, number_experiments)
print("Click through rates are directionally correct in %0.2f fraction of experiments." %fraction_correct)

Let's create a plot of how sample size effects the fraction of experiments with the correct results.

In [ ]:
ctr_A = 0.05
ctr_B = 0.06
number_experiments = 10000

sample_sizes = [
    ###, ###, ###, ###, ###
]
fraction_correct = [calc_percent_correct(sample_size, ctr_A, ctr_B, number_experiments)
                   for sample_size in sample_sizes]

plt.plot(sample_sizes, fraction_correct, '-o')
plt.xlabel('Sample Size')
plt.ylabel('Percent of Experiments with Correct Directionality')
plt.show()

Now let's see how the difference between the two click through rates affects the fraction of experiments with correct results.

In [ ]:
ctr_A = 0.05
sample_size = 1000
number_experiments = 10000

ctr_B_list = [
    ###, ###, ###, ###, ###
]
fraction_correct = [calc_percent_correct(sample_size, ctr_A, ctr_B, number_experiments)
                   for ctr_B in ctr_B_list]

plt.plot(ctr_B_list, fraction_correct, '-o')
plt.xlabel('CTR for B Variant')
plt.ylabel('Percent of Experiments with Correct Directionality')
plt.show()

What if we have much higher click through rates, but always keep the same **relative** difference (10% lift)?

In [ ]:
sample_size = 500
number_experiments = 10000

ctr_A_list = [
    ###, ###, ###, ###, ###
]
fraction_correct = [calc_percent_correct(sample_size, ctr_A, 1.1* ctr_A, number_experiments)
                   for ctr_A in ctr_A_list]

plt.figure(figsize=(15, 6))
ax = plt.subplot()
plt.plot(ctr_A_list, fraction_correct, '-o')
ax.set_xticks(ctr_A_list)
ax.set_xticklabels(['%0.2f/\n%0.2f' %(ctr_A, 1.1 * ctr_A) for ctr_A in ctr_A_list])
plt.xlabel('CTR for A/B Variants')
plt.ylabel('Percent of Experiments with Correct Directionality')
plt.show()

### Example 5

Let's analyze the results of a simple A/B test of two different ads.  We create a **contingency table** with the number of users who fall into each category.

<table>
    <tr>
        <td></td>
        <th>A</th>
        <th>B</th>
    </tr>
    <tr>
        <th>Did Not Click Ad</th>
        <td>950</td>
        <td>940</td>
    </tr>
    <tr>
        <th>Clicked Ad</th>
        <td>50</td>
        <td>60</td>
    </tr>    
</table>

In [ ]:
from scipy.stats import chi2_contingency

contingency = np.array([
    [
        ###, ###
    ],
    [
        ###, ###
    ]
])

g, pvalue, dof, expctd = chi2_contingency(contingency, correction=False)

print('P-value: %0.4f' %pvalue)

What if the proportions were the same but the sample size was 5 times as large?

In [ ]:
contingency = np.array([
    [
        ###, ###
    ],
    [
        ###, ###
    ]
])

g, pvalue, dof, expctd = chi2_contingency(contingency, correction=False)

print('P-value: %0.4f' %pvalue)

It's worth noting that when we only have two variants, we could also use a Proportion Z-Test and get the same answers

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

contingency = np.array([
    [
        ###, ###
    ],
    [
        ###, ###
    ]
])

zscore, pvalue = proportions_ztest(contingency[1, :], contingency.sum(axis=0), alternative='two-sided')
print('P-value: %0.4f' %pvalue)